# PySpark data analysis
Based on [this post](https://towardsdatascience.com/beginners-guide-to-pyspark-bbe3b553b79f).

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import col, lit, countDistinct

In [ ]:
spark = SparkSession.builder\
        .master("spark://localhost:7078")\
        .appName("pyspark-data-analysis")\
        .getOrCreate()

### Loading and analysing data structure

In [ ]:
# Loading a csv
rdd_csv = spark.read.csv("./data/stocks_price_final.csv", sep = ",", header = True)

In [ ]:
# Counting number of records
rdd_csv.count()

In [ ]:
# Printing columns and data types identified
rdd_csv.printSchema()

In [ ]:
# Creating the schema to be used for the RRD during file loading
schema = [
    StructField('_c0', IntegerType(), True),
    StructField('symbol', StringType(), True),
    StructField('date', DateType(), True),
    StructField('open', DoubleType(), True),
    StructField('high', DoubleType(), True),
    StructField('low', DoubleType(), True),
    StructField('close', DoubleType(), True),
    StructField('volume', IntegerType(), True),
    StructField('adjusted', DoubleType(), True),
    StructField('market.cap', StringType(), True),
    StructField('sector', StringType(), True),
    StructField('industry', StringType(), True),
    StructField('exchange', StringType(), True),
]

structure = StructType(fields = schema)

In [ ]:
# Loading a csv specifying schema
rdd_csv = spark.read.csv("./data/stocks_price_final.csv", sep = ",", header = True, schema = structure )

In [ ]:
# Printing columns and data types identified, you can also use .schema, .dtypes, .columns
rdd_csv.printSchema()

In [ ]:
# Printing the first 20 records, you can also use .take(n), .head(n), .first()
rdd_csv.show()

### Querying and visualizing data

In [ ]:
# Filtering data
# Transforming to pandas is important for visualization purposes using matplotlib
TSLA = rdd_csv.filter(col("symbol") == lit("TSLA")).toPandas()
GME = rdd_csv.filter(col("symbol") == lit("GME")).toPandas()

In [ ]:
fig, axes = plt.subplots(2, 1, sharex = True, figsize = (30, 15))
axes[0].plot(TSLA["date"], TSLA["high"], label = "Highest price")
axes[0].plot(TSLA["date"], TSLA["low"], label = "Lowest price")
axes[1].plot(GME["date"], GME["high"], label = "Highest price")
axes[1].plot(GME["date"], GME["low"], label = "Lowest price")
axes[0].set_title("Stock price time series for TSLA and GME")
axes[0].set_ylabel("Price")
axes[1].set_ylabel("Price")
plt.xlabel("Date")
axes[0].grid()
axes[1].grid()
axes[0].legend()
axes[1].legend()

In [ ]:
ncomp_by_sector = rdd_csv.select(["symbol", "sector"]).groupBy("sector").agg(countDistinct("symbol").alias("n_companies")).orderBy(col("n_companies")).toPandas()

In [ ]:
plt.figure(figsize = (15, 10))
plt.barh(ncomp_by_sector["sector"], ncomp_by_sector["n_companies"])

### Storing data

In [ ]:
# Storing data in JSON format
rdd_csv.filter(col("symbol") == lit("TSLA")).select(["date", "high", "low"]).write.save("./data/TSLA_timeseries.csv", format = "csv", header = "true")

### Stoping Spark context and session

In [ ]:
spark.stop()